In [1]:
from datasets import load_dataset, interleave_datasets
from datasets import Dataset
import pandas as pd

# Load in the component datasets to be combined

In [2]:
health_assistant_dataset = load_dataset('alanahmet/HealthAssistant115')['train']
mental_health_dataset = load_dataset('Amod/mental_health_counseling_conversations')['train']
gym_dataset = load_dataset('garyzsu/custom_gym_dataset')['train']
workouts_dataset = load_dataset('Kolibri753/generate-workouts')['train']

/Users/michaelruberto/opt/anaconda3/envs/LLM/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


# Process each dataset as necessary so they all have the same format

In [3]:
health_assistant_dataset = health_assistant_dataset.rename_column('instruction', 'input')
health_assistant_dataset

Dataset({
    features: ['output', 'input'],
    num_rows: 230
})

In [4]:
mental_health_dataset = mental_health_dataset.rename_column('Context', 'input')
mental_health_dataset = mental_health_dataset.rename_column('Response', 'output')
mental_health_dataset = Dataset.from_pandas(pd.DataFrame(mental_health_dataset).drop_duplicates(subset='input').set_index('input'))
mental_health_dataset

Dataset({
    features: ['output', 'input'],
    num_rows: 995
})

In [5]:
gym_dataset = gym_dataset.rename_column('question', 'input')
gym_dataset = gym_dataset.rename_column('answer', 'output')
gym_dataset = gym_dataset.remove_columns(['context', 'category'])
gym_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 74
})

In [6]:
workouts_dataset = workouts_dataset.add_column('input', [text[text.index('"')+1:text.index('" ')] for text in workouts_dataset['text']])
workouts_dataset = workouts_dataset.add_column('output', [text[text.index('[/INST] ')+8:text.index(' </s>')] for text in workouts_dataset['text']])
workouts_dataset = workouts_dataset.remove_columns(['text'])
workouts_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 101
})

# Merge into one dataset and push to hub

In [7]:
merged_dataset = interleave_datasets([workouts_dataset, gym_dataset, mental_health_dataset, health_assistant_dataset],
                                     stopping_strategy="all_exhausted")
merged_dataset = Dataset.from_pandas(pd.DataFrame(merged_dataset).drop_duplicates(subset='input').set_index('input'))
merged_dataset = merged_dataset.train_test_split(test_size=0.21)
merged_dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'input'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['output', 'input'],
        num_rows: 266
    })
})

In [8]:
# merged_dataset.push_to_hub('rubertmi00/HealthCoachDataset')